In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import keras

2024-03-02 17:29:32.134135: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-02 17:29:32.134190: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-02 17:29:32.135699: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam, SGD, RMSprop
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.layers import Flatten, Dense
from keras.models import Model

In [ ]:
!wget https://www.dropbox.com/s/w3zlhing4dkgeyb/train.zip?dl=0


In [ ]:
!unzip train.zip?dl=0


In [ ]:
model1 = MobileNet(input_shape=(224,224,3), include_top=False)

for layer in model1.layers:
  layer.trainable = False

x = Flatten()(model1.output)
x = Dense(units=7, activation='softmax')(x)

model = Model(model1.input, x)

In [ ]:
model.summary()



In [ ]:
model.compile(optimizer='adam', loss="categorial_crossentropy", metrics=['accuracy'])


In [ ]:
train_datagen = ImageDataGenerator(
    zoom_range = 0.2,
    shear_range = 0.2,
    horizontal_flip = True,
    rescale = 1./255
)
train_data = train_datagen.flow_from_directory(directory="../input/facedata/train",
                                               target_size=(224,224),
                                               batch_size=32)
train_data.class_indices

In [ ]:
val_datagen = ImageDataGenerator(rescale = 1/255)

val_data = val_datagen.flow_from_directory(directory="../input/facedata/train",
                                           target_size=(224,224),
                                           batch_size=32)

In [ ]:
import matplotlib.pyplot as plt

# Assuming train_data is already defined as a DirectoryIterator

t_img, label = next(train_data)

def plotImages(img_arr, label):
    """
    input :- images array
    output :- plot the images
    """

    count = 0
    for im, l in zip(img_arr, label):
        plt.imshow(im)
        plt.title(im.shape)
        plt.axis('off')  # Corrected line to turn off axis
        plt.show()

        count += 1
        if count == 10:
            break

plotImages(t_img, label)

     

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

es = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=5, verbose=1, mode = 'auto')

mc = ModelCheckpoint(filepath = "model1.keras", monitor = 'val_accuracy', verbose=1, save_best_only = True, mode = 'auto')
call_back = [es, mc]



In [ ]:
from keras.losses import categorical_crossentropy

model.compile(loss=categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(train_data,
                           steps_per_epoch=10,
                           epochs=30,
                           validation_data=val_data,
                           validation_steps=8,
                           callbacks=[es,mc])

In [ ]:
from keras.models import load_model
model=load_model("/kaggle/working/model1.keras")

In [ ]:


h = history.history
h.keys()

In [ ]:

plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'])
plt.title("accuracy vs validation-accuracy")
plt.legend(['train','test'],loc ='upper left')
plt.show()


In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'], c = "red")
plt.title("loss vs validation-loss")
plt.show()

In [ ]:
import tensorflow as tf

# Load the Keras model from the .keras file
model = tf.keras.models.load_model("/kaggle/working/model1.keras")

# Save the model to a .h5 file
model.save("model.h5")
